# TITLE OF PROJECT

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time:
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:
* Video of 5-min Non-Technical Presentation:

## TABLE OF CONTENTS 

*Click to jump to matching Markdown Header.*<br><br>
 
- **[Introduction](#INTRODUCTION)<br>**
- **[OBTAIN](#OBTAIN)**<br>
- **[SCRUB](#SCRUB)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
___

# INTRODUCTION

> Explain the point of your project and what question you are trying to answer with your modeling.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***



# OBTAIN

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***


In [1]:
# import pandas as pd
# from sodapy import Socrata

# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# # client = Socrata("data.wa.gov", None)

# # Example authenticated client (needed for non-public datasets):
# client = Socrata('data.wa.gov',
#                  'C7QFByRWjTCvFfagF5Lt1QE50',
#                  username='427yub1zw87tmeol5eia062z',
#                  password='23yxf9ytpdtln3u8l4lxfv9u3y7gjidxh8kkw383xv7gbd4ogq')

# # First 2000 results, returned as JSON from API / converted to Python list of
# # dictionaries by sodapy.
# results = client.get("rpr4-cgyd", limit=2000)

# # Convert to pandas DataFrame
# results_df = pd.DataFrame.from_records(results)

In [2]:
run_api_calls = False

In [3]:
#defining a function to open/load json files
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [4]:
#storing api keys in variables
keys = get_keys("/Users/berke/.secret/socrata_api_project_5.json")

api_key_socrata = keys['api_key']
app_token_socrata = keys['app_token']
api_key_secret_socrata = keys['api_key_secret']

In [5]:
import requests
import time
import pandas as pd

if run_api_calls == True:
    #Defining/initializing API request headers and parameters
    headers = {'X-App-Token': app_token_socrata, 'username': api_key_socrata,
               'password': api_key_secret_socrata}
    params = {'$limit': '50000', '$offset': None}
    offset = list(range(0, 500000, 50000))

    #Requesting data from API and parsing results to a dictionary
    dfs = {}
    for number in offset:
        params['$offset'] = str(number)
        r = requests.get(f'https://data.wa.gov/resource/rpr4-cgyd.json?', 
                         headers=headers, params=params)
        dfs[f'df_{number}']=pd.DataFrame.from_records(r.json())
        time.sleep(1)

In [6]:
if run_api_calls == True:
    #saving all pages as one csv file
    import os
    path = './data'
    output_file = os.path.join(path,'title_transactions.csv.gz')

    for key, df in dfs.items():
        if key == 'df_0':
            df_final = dfs['df_0']
        else:
            df_final = pd.concat([df_final, df], axis=0)

    df_final.to_csv(output_file, index=False, compression='gzip')    

In [7]:
#importing the data back into the notebook
df=pd.read_csv('data/title_transactions.csv.gz', compression='gzip', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433172 entries, 0 to 433171
Data columns (total 30 columns):
 #   Column                                                   Non-Null Count   Dtype  
---  ------                                                   --------------   -----  
 0   electric_vehicle_type                                    433172 non-null  object 
 1   vin_1_10                                                 433172 non-null  object 
 2   model_year                                               433172 non-null  int64  
 3   make                                                     433172 non-null  object 
 4   model                                                    433172 non-null  object 
 5   new_or_used_vehicle                                      433172 non-null  object 
 6   sale_price                                               433172 non-null  float64
 7   transaction_date                                         433172 non-null  object 
 8   transaction_ty

# SCRUB

In [8]:
pd.set_option('max_columns', None)
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_date,transaction_type,transaction_year,electric_vehicle_fee_paid,county,city,zip,electric_range,base_msrp,non_clean_alternative_fuel,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,legislative_district,hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility,meets_2019_hb_2042_electric_range_requirement,meets_2019_hb_2042_sale_date_requirement,meets_2019_hb_2042_sale_price_value_requirement,odometer_reading,odometer_code,transportation_electrification_fee_paid,hybrid_vehicle_electrification_fee_paid
0,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,2017-01-31T00:00:00.000,Original Title,2017,Not Applicable,King,MEDINA,98039.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2016-12-27T00:00:00.000,Passenger,WA,348693390,48.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
1,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,2017-03-22T00:00:00.000,Original Registration,2017,No,King,SNOQUALMIE,98065.0,200,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,329748177,5.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,2017-03-22T00:00:00.000,Original Title,2017,Not Applicable,King,SNOQUALMIE,98065.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-03T00:00:00.000,Passenger,WA,329748177,5.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
3,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,2017-03-24T00:00:00.000,Original Registration,2017,No,Island,OAK HARBOR,98277.0,53,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,242531271,10.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
4,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,2017-03-24T00:00:00.000,Original Title,2017,Not Applicable,Island,OAK HARBOR,98277.0,53,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-11T00:00:00.000,Passenger,WA,242531271,10.0,TRANSACTION NOT ELIGIBLE: Sale before August 2019,True,False,True,99,Actual Mileage,Not Applicable,Not Applicable


In [9]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_date,transaction_type,transaction_year,electric_vehicle_fee_paid,county,city,zip,electric_range,base_msrp,non_clean_alternative_fuel,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,legislative_district,hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility,meets_2019_hb_2042_electric_range_requirement,meets_2019_hb_2042_sale_date_requirement,meets_2019_hb_2042_sale_price_value_requirement,odometer_reading,odometer_code,transportation_electrification_fee_paid,hybrid_vehicle_electrification_fee_paid
0,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,2017-01-31,Original Title,2017,Not Applicable,King,MEDINA,98039.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2016-12-27T00:00:00.000,Passenger,WA,348693390,48.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
1,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,2017-03-22,Original Registration,2017,No,King,SNOQUALMIE,98065.0,200,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,329748177,5.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,2017-03-22,Original Title,2017,Not Applicable,King,SNOQUALMIE,98065.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-03T00:00:00.000,Passenger,WA,329748177,5.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
3,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,2017-03-24,Original Registration,2017,No,Island,OAK HARBOR,98277.0,53,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,242531271,10.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
4,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,2017-03-24,Original Title,2017,Not Applicable,Island,OAK HARBOR,98277.0,53,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-11T00:00:00.000,Passenger,WA,242531271,10.0,TRANSACTION NOT ELIGIBLE: Sale before August 2019,True,False,True,99,Actual Mileage,Not Applicable,Not Applicable


In [10]:
df.set_index('transaction_date', inplace=True)
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,transaction_year,electric_vehicle_fee_paid,county,city,zip,electric_range,base_msrp,non_clean_alternative_fuel,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,legislative_district,hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility,meets_2019_hb_2042_electric_range_requirement,meets_2019_hb_2042_sale_date_requirement,meets_2019_hb_2042_sale_price_value_requirement,odometer_reading,odometer_code,transportation_electrification_fee_paid,hybrid_vehicle_electrification_fee_paid
transaction_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-31,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,Original Title,2017,Not Applicable,King,MEDINA,98039.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2016-12-27T00:00:00.000,Passenger,WA,348693390,48.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,Original Registration,2017,No,King,SNOQUALMIE,98065.0,200,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,329748177,5.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,Original Title,2017,Not Applicable,King,SNOQUALMIE,98065.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-03T00:00:00.000,Passenger,WA,329748177,5.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,Original Registration,2017,No,Island,OAK HARBOR,98277.0,53,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,242531271,10.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,Original Title,2017,Not Applicable,Island,OAK HARBOR,98277.0,53,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-11T00:00:00.000,Passenger,WA,242531271,10.0,TRANSACTION NOT ELIGIBLE: Sale before August 2019,True,False,True,99,Actual Mileage,Not Applicable,Not Applicable


In [11]:
df[df['vin_1_10']=='Registration Renewal'].head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,transaction_year,electric_vehicle_fee_paid,county,city,zip,electric_range,base_msrp,non_clean_alternative_fuel,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,legislative_district,hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility,meets_2019_hb_2042_electric_range_requirement,meets_2019_hb_2042_sale_date_requirement,meets_2019_hb_2042_sale_price_value_requirement,odometer_reading,odometer_code,transportation_electrification_fee_paid,hybrid_vehicle_electrification_fee_paid
transaction_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [12]:
df[df['vin_1_10']=='5YJSA1DP3D']

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,transaction_year,electric_vehicle_fee_paid,county,city,zip,electric_range,base_msrp,non_clean_alternative_fuel,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,legislative_district,hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility,meets_2019_hb_2042_electric_range_requirement,meets_2019_hb_2042_sale_date_requirement,meets_2019_hb_2042_sale_price_value_requirement,odometer_reading,odometer_code,transportation_electrification_fee_paid,hybrid_vehicle_electrification_fee_paid
transaction_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-21,Battery Electric Vehicle (BEV),5YJSA1DP3D,2013,TESLA,MODEL S,Used,0.0,Registration Renewal,2017,Yes,King,KIRKLAND,98034.0,208,69900,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,223757775,1.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2017-04-19,Battery Electric Vehicle (BEV),5YJSA1DP3D,2013,TESLA,MODEL S,Used,0.0,Registration at time of Transfer,2017,No,Snohomish,STANWOOD,98292.0,208,69900,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,145132585,10.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2017-04-19,Battery Electric Vehicle (BEV),5YJSA1DP3D,2013,TESLA,MODEL S,Used,0.0,Transfer Title,2017,Not Applicable,Snohomish,STANWOOD,98292.0,208,69900,TRANSACTION NOT ELIGIBLE: Not a new vehicle; V...,2017-04-19T00:00:00.000,Passenger,WA,145132585,10.0,TRANSACTION NOT ELIGIBLE: Sales and Use Tax Ex...,True,False,False,52268,Actual Mileage,Not Applicable,Not Applicable
2017-09-21,Battery Electric Vehicle (BEV),5YJSA1DP3D,2013,TESLA,MODEL S,Used,0.0,Registration Renewal,2017,Yes,King,REDMOND,98052.0,208,69900,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,193060787,45.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2018-06-21,Battery Electric Vehicle (BEV),5YJSA1DP3D,2013,TESLA,MODEL S,Used,0.0,Registration Renewal,2018,Yes,King,SNOQUALMIE,98065.0,208,69900,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,147602080,5.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-14,Battery Electric Vehicle (BEV),5YJSA1DP3D,2013,TESLA,MODEL S,Used,0.0,Registration Renewal,2020,Yes,Kitsap,PORT ORCHARD,98366.0,208,69900,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,235000580,26.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,Yes,No
2019-10-31,Battery Electric Vehicle (BEV),5YJSA1DP3D,2013,TESLA,MODEL S,Used,0.0,Registration at time of Transfer,2019,Yes,King,SNOQUALMIE,98065.0,208,69900,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,186196202,5.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,Yes,No
2019-10-31,Battery Electric Vehicle (BEV),5YJSA1DP3D,2013,TESLA,MODEL S,Used,39000.0,Transfer Title,2019,Not Applicable,King,SNOQUALMIE,98065.0,208,69900,TRANSACTION NOT ELIGIBLE: Not a new vehicle; T...,2019-09-24T00:00:00.000,Passenger,WA,186196202,5.0,TRANSACTION NOT ELIGIBLE: Sale price too high,True,True,False,38233,Actual Mileage,Not Applicable,Not Applicable


In [13]:
drop_cols = ['transaction_year', 'electric_vehicle_fee_paid', 'base_msrp', 
             'non_clean_alternative_fuel', 'date_of_vehicle_sale',
             'hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility',
             'meets_2019_hb_2042_electric_range_requirement', 
             'meets_2019_hb_2042_sale_date_requirement', 
             'meets_2019_hb_2042_sale_price_value_requirement', 
             'transportation_electrification_fee_paid', 
             'hybrid_vehicle_electrification_fee_paid', 'legislative_district']
df.drop(drop_cols, axis=1, inplace=True)
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code
transaction_date,,,,,,,,,,,,,,,,,
2017-01-31,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,Original Title,King,MEDINA,98039.0,200,Passenger,WA,348693390,50,Actual Mileage
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,Original Registration,King,SNOQUALMIE,98065.0,200,Passenger,WA,329748177,0,Odometer reading is not collected at time of r...
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,Original Title,King,SNOQUALMIE,98065.0,200,Passenger,WA,329748177,50,Actual Mileage
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,Original Registration,Island,OAK HARBOR,98277.0,53,Passenger,WA,242531271,0,Odometer reading is not collected at time of r...
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,Original Title,Island,OAK HARBOR,98277.0,53,Passenger,WA,242531271,99,Actual Mileage


In [14]:
df['electric_vehicle_type'].value_counts()

Battery Electric Vehicle (BEV)            302756
Plug-in Hybrid Electric Vehicle (PHEV)    130416
Name: electric_vehicle_type, dtype: int64

In [15]:
df['new_or_used_vehicle'].value_counts()

Used    275022
New     158150
Name: new_or_used_vehicle, dtype: int64

In [16]:
df['vehicle_primary_use'].value_counts()

Passenger                             429023
Exempt (State/County/Local/Tribal)      2566
Commercial                               755
Truck                                    615
For Hire                                 134
Medium-Speed Electric Vehicle             47
Neighborhood Electric Vehicle             13
Cab                                        5
Taxi Cab                                   3
Medium-Speed Electric Truck                3
Farm Use                                   3
Private School                             3
Farm Exempt                                2
Name: vehicle_primary_use, dtype: int64

In [17]:
df['state_of_residence'].value_counts()

WA    432652
CA        86
VA        68
TX        53
OR        41
MD        28
GA        25
MI        18
NV        15
IL        14
NY        14
NJ        14
CO        13
FL        12
AK        12
CT        10
NC        10
OK         8
AL         8
KS         7
MA         6
AZ         6
DC         5
ID         5
UT         5
MT         4
HI         4
SC         3
OH         3
TN         3
LA         3
MN         2
AR         2
KY         2
DE         2
MO         1
WY         1
PA         1
IN         1
WI         1
Name: state_of_residence, dtype: int64

In [18]:
df['make'].value_counts()

TESLA                   133069
NISSAN                  109494
CHEVROLET                57133
FORD                     31540
BMW                      20725
KIA                      19929
TOYOTA                   18223
HYUNDAI                   6384
AUDI                      5858
CHRYSLER                  4786
FIAT                      4282
VOLKSWAGEN                4222
VOLVO                     3375
HONDA                     3133
MITSUBISHI                2408
PORSCHE                   1976
SMART                     1968
MERCEDES-BENZ             1358
MINI                       987
CADILLAC                   779
JAGUAR                     630
FISKER                     252
SUBARU                     152
JEEP                       118
LAND ROVER                 115
AZURE DYNAMICS              71
WHEEGO ELECTRIC CARS        61
LINCOLN                     60
TH!NK                       46
POLESTAR                    20
DODGE                        7
BUBBLE BUDDY                 6
PLYMOUTH

In [28]:
df['transaction_type'].value_counts()

Registration Renewal                191199
Original Registration                97449
Original Title                       96961
Transfer Title                       24581
Registration at time of Transfer     22982
Name: transaction_type, dtype: int64

In [46]:
df = df[(df['transaction_type']=='Original Title') | (df['transaction_type']=='Transfer Title')]

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 121542 entries, 2017-01-31 to 2017-05-01
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   electric_vehicle_type  121542 non-null  object 
 1   vin_1_10               121542 non-null  object 
 2   model_year             121542 non-null  int64  
 3   make                   121542 non-null  object 
 4   model                  121542 non-null  object 
 5   new_or_used_vehicle    121542 non-null  object 
 6   sale_price             121542 non-null  float64
 7   transaction_type       121542 non-null  object 
 8   county                 121534 non-null  object 
 9   city                   121514 non-null  object 
 10  zip                    121537 non-null  float64
 11  electric_range         121542 non-null  int64  
 12  vehicle_primary_use    121542 non-null  object 
 13  state_of_residence     121540 non-null  object 
 14  dol_vehicle_id      

In [49]:
len(df['dol_vehicle_id'].unique())

91598

In [52]:
df['county'].value_counts().head(20)

King            66136
Snohomish       13087
Pierce           9501
Clark            6421
Thurston         4334
Kitsap           4301
Whatcom          3127
Spokane          2714
Benton           1339
Island           1325
Skagit           1295
Clallam           754
Jefferson         732
San Juan          713
Mason             570
Cowlitz           567
Chelan            564
Yakima            539
Lewis             462
Grays Harbor      424
Name: county, dtype: int64

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

# EXPLORE

# MODEL

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

# iNTERPRET

Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

# CONCLUSIONS & RECOMMENDATIONS

Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***